# Import

In [2]:
import numpy as np

# Define

In [71]:
def guess_square_error(input_answers, input_weights = None):
    rowwise_mean = np.average(input_answers, axis = 0)
    for a in range(input_answers.shape[0]):
        input_answers[a] -= rowwise_mean

    square_ans = input_answers * input_answers
    if input_weights != None:
        for a in range(input_answers.shape[0]):
            square_ans[a] *= input_weights[a]

    return square_ans.sum()

class VariableMatrix():
    def __init__(self, size, mean = 0, std = 1, work_components = 1, is_independent_learning = False):
        self.size = size
        self.values = np.random.normal(mean, std, size)
        self.diff = np.zeros(size)
        self.drop = np.ones(size, dtype = bool)
        
        if work_components == 1:
            self.work = np.ones(size, dtype = bool)
        else:
            self.work = (np.random.random(size) < work_components)
        
        if is_independent_learning:
            self.eta_m = np.zeros(size)
        else:
            self.eta_m = None

    def move(self, eta = None):
        move_values = (-1) * self.diff * self.works * self.drop
        if eta == None:
            self.values += self.eta_m * move_values
        else:
            self.values += eta * move_values
            
class Sigmoid():
    def trans(self, x):
        out_put = np.zeros([len(x)])
        for i in range(len(x)):
            out_put[i] = 1 / (1 + np.exp(-x[i]))
        
        return out_put
    
    def diff(self, x):
        out_put = np.zeros([len(x), len(x)])
        for i in range(len(x)):
            out_put[i][i] = 1 / (1 + np.exp(x[i]) * (1 + np.exp(-x[i])))
            
        return out_put
    
class Identity():
    def trans(self, x):
        return x
    
    def diff(self, x):
        return np.identity(len(x))

class Layer():
    def __init__(self, layers_from, nodes_n, active_function):
        self.lf = layers_from # list :layers connected before
        self.lt = [] # layers connected after
        self.nodes_n = nodes_n
        self.weights = {}
        self.input_bias = VariableMatrix([self.nodes_n])
        self.nodes_input = np.zeros([self.nodes_n])
        self.nodes_output = np.zeros([self.nodes_n])
        self.nodes_diff = np.zeros([self.nodes_n, self.nodes_n])
        self.af = active_function
            
    def forward(self):
        self.nodes_output = self.af.trans(self.nodes_input)

    def jacobi(self):
        self.nodes_diff = self.af.diff(self.nodes_input)
    
class DogikoNN():
    def __init__(self):
        self.pp_linear = {
            "multi" : None,
            "trans" : None
        } # pre-processing linear translation
        self.flow = []
        self.layers = {}
    
    def set_training_data(self, input_datas, input_answers, input_weights = None):
        # type input_data : numpy array
        # type input_answer : numpy array
        self.t_datas = input_datas
        self.t_answers = input_answers
        self.t_weights = input_weights
        self.data_size = self.t_datas.shape[1:]
        self.answer_size = self.t_answers.shape[1:]
    
    def set_validating_data(self, input_datas, input_answers, input_weights = None):
        # type input_data : numpy array
        # type input_answer : numpy array
        if input_datas.shape[1:] != self.data_size:
            print ("Size of datas should be the same as training data, must set_training_data before set_validating_data!")

        if input_answers.shape[1:] != self.answer_size:
            print ("Size of answers should be the same as training data, must set_training_data before set_validating_data!")
            
        self.v_datas = input_datas
        self.v_answers = input_answers
        self.v_weights = input_weights
        
        
    def define_normalized(self, normalized_algorithm):
        self.normal_alg = normalized_algorithm
    
    def normalized(self, input_datas):
        out_put = np.zeros(input_datas.shape)
        for d in range(len(input_datas)):
            out_put[d] = self.pp_linear["multi"] * (self.t_datas[d] - self.pp_linear["trans"])
        
        return out_put
    
    def layers_clear(self):
        self.layers = {}
        
    def insert_layer(self, name, layer, is_flow_add = True):
        self.layers[name] = layer
        if is_flow_add:
            self.flow.append(name)
        
    def build(self):
        if self.normal_alg == "normal":
            temp_trans_datas = self.t_datas.T
            self.single_data_size = len(temp_trans_datas)
            temp_square_mean_array = np.zeros([self.single_data_size])
            temp_sd = np.zeros([self.single_data_size])
            self.pp_linear["trans"] = np.zeros([self.single_data_size])
            for j in range(self.single_data_size):
                feature = temp_trans_datas[j]
                self.pp_linear["trans"][j] = feature.mean()
                temp_sd[j] = feature.std()
            
            temp_sd[temp_sd == 0] = 1
            self.pp_linear["multi"] = 1 / temp_sd
            
        else:
            print ("unacceptable normalized_algorithm")
            return ("unacceptable normalized_algorithm")
        
        self.n_t_datas = self.normalized(self.t_datas)
        self.n_v_datas = self.normalized(self.v_datas)
        
        if len(self.flow) != len(set(self.flow)):
            print ("each element in .flow should be unique (layer name)")
            return ("each element in .flow should be unique (layer name)")
        
        for l in range(len(self.flow)):
            ln = self.flow[l]
            self.layers[ln].lt = []
            for ln2 in self.layers[ln].lf:
                if ln2 == "input_source":
                    self.layers[ln].weights[ln2] = VariableMatrix([self.layers[ln].nodes_n, self.single_data_size])
                elif ln2 not in self.layers:
                    print ("layer " + ln2 + " doesn't exist!")
                else:
                    if self.flow.index(ln2) > l:
                        print ("layer " + ln2 + " shoud work before layer " + ln)
                        return ("layer " + ln2 + " shoud work before layer " + ln)
                    
                    self.layers[ln2].lt.append(ln)
                    self.layers[ln].weights[ln2] = VariableMatrix([self.layers[ln].nodes_n, self.layers[ln2].nodes_n])
        
    def forward_propagation(self, input_array):
        for l in range(len(self.flow)):
            ln = self.flow[l]
            self.layers[ln].nodes_input *= 0.
            for ln2 in self.layers[ln].lf:
                if ln2 == "input_source":
                    self.layers[ln].nodes_input += self.layers[ln].weights[ln2].values.dot(input_array)
                else:
                    self.layers[ln].nodes_input += self.layers[ln].weights[ln2].values.dot(self.layers[ln2].nodes_output)
                    
            self.layers[ln].nodes_input += self.layers[ln].input_bias.values
            self.layers[ln].forward()
            self.layers[ln].jacobi()
            
        return self.layers[self.flow[-1]].nodes_output
    
    def prediction(self, input_data):
        out_put = np.zeros(input_data.shape[:1] + self.answer_size)
        for d in range(input_data.shape[0]):
            out_put[d] = self.forward_propagation(input_data[d])
        
        return out_put
    
    def backward_propagation(self):
        for l in range(len(self.flow) - 1, -1, -1):
            ln = self.flow[l]
    
    
    

In [62]:
try_NN = DogikoNN()

try_data = np.random.rand(*[3, 2]) * 2 - 1
try_answer = np.random.randint(2, size = [3, 2])
try_NN.set_training_data(try_data, try_answer)
try_NN.set_validating_data(try_data, try_answer)

try_NN.define_normalized("normal")

try_NN.insert_layer(
    'A',
    Layer(
        layers_from = ["input_source"],
        nodes_n = 2,
        active_function = Sigmoid()
    )
)

try_NN.insert_layer(
    'B',
    Layer(
        layers_from = ['A'],
        nodes_n = 3,
        active_function = Sigmoid()
    )
)

try_NN.insert_layer(
    'C',
    Layer(
        layers_from = ['A'],
        nodes_n = 4,
        active_function = Sigmoid()
    )
)

try_NN.insert_layer(
    'D',
    Layer(
        layers_from = ['B', 'C'],
        nodes_n = 2,
        active_function = Sigmoid()
    )
)

In [63]:
try_NN.build()

In [4]:
np.array(range(4,-1,-1))

array([4, 3, 2, 1, 0])